In [ ]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# Question 1: [Macro] Average growth of GDP in 2023

**What is the average growth (in %) of GDP in 2023?**

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7

In [ ]:
gdpc1 = pdr.DataReader("GDPC1", "fred")

In [ ]:
gdpc1.tail()

,GDPC1
DATE,
2022-10-01,21989.981
2023-01-01,22112.329
2023-04-01,22225.350
2023-07-01,22490.692
2023-10-01,22679.255


In [ ]:
gdpc1["gdpc1_yoy"] = gdpc1.GDPC1 / gdpc1.GDPC1.shift(4) - 1

In [ ]:
gdpc1_2023 = gdpc1[gdpc1.index >= "2023-01-01"]

In [ ]:
avg_gdpc1_2023 = gdpc1_2023["gdpc1_yoy"].sum() / len(gdpc1_2023["gdpc1_yoy"]) * 100

In [ ]:
print(f"Average GDPC1 in 2023 is {round(avg_gdpc1_2023, 1)}%")

Average GDPC1 in 2023 is 2.5%


# Question 2. [Macro] Inverse "Treasury Yield"

**Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.**

Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [ ]:
start_date = "2000-01-01"

In [ ]:
dgs10 = pdr.DataReader("DGS10", "fred", start=start_date)
dgs2 = pdr.DataReader("DGS2", "fred", start=start_date)
dgs = dgs10.join(dgs2)
dgs["difference"] = dgs.DGS10 - dgs.DGS2
dgs.head()

,DGS10,DGS2,difference
DATE,,,
2000-01-03,6.58,6.38,0.20
2000-01-04,6.49,6.30,0.19
2000-01-05,6.62,6.38,0.24
2000-01-06,6.57,6.35,0.22
2000-01-07,6.52,6.31,0.21


In [ ]:
print(f"Minimun difference between DGS10 - DGS2 is {round(dgs['difference'].min(), 1)}")

Minimun difference between DGS10 - DGS2 is -1.1


# Question 3. [Index] Which Index is better recently?

**Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)**

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

In [ ]:
SnP500 = yf.download(tickers = "^GSPC")
SnP500["Close 5 YG"] = SnP500["Close"] / SnP500["Close"].shift(252*5) - 1
SnP500 = SnP500[["Close", "Close 5 YG"]]
SnP500.tail()

[*********************100%%**********************]  1 of 1 completed


,Close,Close 5 YG
Date,,
2024-04-16,5051.410156,0.737426
2024-04-17,5022.209961,0.728471
2024-04-18,5011.120117,0.723776
2024-04-19,4967.229980,0.712572
2024-04-22,5010.600098,0.724801


In [ ]:
IPC_mexico = yf.download(tickers = "^MXX")
IPC_mexico["Close 5 YG"] = IPC_mexico["Close"] / IPC_mexico["Close"].shift(252*5) - 1
IPC_mexico = IPC_mexico[["Close", "Close 5 YG"]]
IPC_mexico.tail()

[*********************100%%**********************]  1 of 1 completed


,Close,Close 5 YG
Date,,
2024-04-16,55797.250000,0.248650
2024-04-17,55415.691406,0.241786
2024-04-18,55739.429688,0.238396
2024-04-19,55862.851562,0.227073
2024-04-22,56551.898438,0.246152


In [ ]:
start = "2019-04-09"
end = "2024-04-09"

SnP500_subset = SnP500[(SnP500.index >= start) & (SnP500.index <= end)]
IPC_mexico_subset = IPC_mexico[(IPC_mexico.index >= start) & (IPC_mexico.index <= end)]

In [ ]:
print(f"SnP500_subset max \t= {int(SnP500_subset['Close 5 YG'].max() * 100)}")
print(f"IPC_mexico_subset max \t= {int(IPC_mexico_subset['Close 5 YG'].max() * 100)}")

SnP500_subset max 	= 125
IPC_mexico_subset max 	= 43


# Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks

**Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023**

Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

In [ ]:
start = "2023-01-01"
end = "2023-12-31"

In [ ]:
SR2222 = yf.download(tickers = "2222.SR",
                     period = "max",
                     interval = "1d",
                     start = start,
                     end = end)
BRK_B = yf.download(tickers = "BRK-B",
                     period = "max",
                     interval = "1d",
                     start = start,
                     end = end)
AAPL = yf.download(tickers = "AAPL",
                     period = "max",
                     interval = "1d",
                     start = start,
                     end = end)
MSFT = yf.download(tickers = "MSFT",
                     period = "max",
                     interval = "1d",
                     start = start,
                     end = end)
GOOG = yf.download(tickers = "GOOG",
                     period = "max",
                     interval = "1d",
                     start = start,
                     end = end)
JPM = yf.download(tickers = "JPM",
                     period = "max",
                     interval = "1d",
                     start = start,
                     end = end)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
def calculate(data):
  max = data.max()
  min = data.min()
  return round((max-min)/max, 2)

In [ ]:
print(f"""
SR2222\t: {calculate(SR2222['Adj Close'])}
BRK_B\t: {calculate(BRK_B['Adj Close'])}
AAPL\t: {calculate(AAPL['Adj Close'])}
MSFT\t: {calculate(MSFT['Adj Close'])}
GOOG\t: {calculate(GOOG['Adj Close'])}
JPM\t: {calculate(JPM['Adj Close'])}
      """)


SR2222	: 0.21
BRK_B	: 0.21
AAPL	: 0.37
MSFT	: 0.42
GOOG	: 0.39
JPM	: 0.28
      


# Question 5. [Stocks] Dividend Yield

**Find the largest dividend yield for the same set of stocks**

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023. You can use get_actions() method or .dividends field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximum value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

In [ ]:
SR2222_div = yf.Ticker("2222.SR").dividends
BRK_B_div = yf.Ticker("BRK-B").dividends
AAPL_div = yf.Ticker("AAPL").dividends
MSFT_div = yf.Ticker("MSFT").dividends
GOOG_div = yf.Ticker("GOOG").dividends
JPM_div = yf.Ticker("JPM").dividends

In [ ]:
def sum_dividends(series):
  start = "2023-01-01"
  end = "2023-12-31"
  return series[(series.index > start) & (series.index < end)].sum()

In [ ]:
print("Dividends yields in 2023\n")

DY_SR2222 = round(sum_dividends(SR2222_div) / SR2222["Adj Close"][-1] * 100, 1)
DY_BRK_B = round(sum_dividends(BRK_B_div) / BRK_B["Adj Close"][-1] * 100, 1)
DY_AAPL = round(sum_dividends(AAPL_div) / AAPL["Adj Close"][-1] * 100, 1)
DY_MSFT = round(sum_dividends(MSFT_div) / MSFT["Adj Close"][-1] * 100, 1)
DY_GOOG = round(sum_dividends(GOOG_div) / GOOG["Adj Close"][-1] * 100, 1)
DY_JPM = round(sum_dividends(JPM_div) / JPM["Adj Close"][-1] * 100, 1)

print(f"SR2222 \t= {DY_SR2222}")
print(f"BRK_B \t= {DY_BRK_B}")
print(f"AAPL \t= {DY_AAPL}")
print(f"MSFT \t= {DY_MSFT}")
print(f"GOOG \t= {DY_GOOG}")
print(f"JPM \t= {DY_JPM}")

Dividends yields in 2023

SR2222 	= 2.8
BRK_B 	= 0.0
AAPL 	= 0.5
MSFT 	= 0.7
GOOG 	= 0.0
JPM 	= 2.4
